<a href="https://colab.research.google.com/github/yoonkim313/dataCampusProject-Team10/blob/master/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

!git clone https://github.com/yoonkim313/dataCampusProject-Team10.git

    ! python3 /content/drive/Shared drives/BigDATA TEAM 10/dataCampusProject-Team10/ocr/deep-text-recognition/train.py
    ! python /content/drive/Shared drives/BigDATA TEAM 10/dataCampusProject-Team10/main.ipynb
    sys.path.insert(0,nb_path)
    !pip install --target=$nb_path transformers
    !apt-get update
    !apt-get g++ openjdk-8-jdk 
    !pip3 install --target=$nb_path konlpy
    !pip install --target=$nb_path soykeyword
    !pip install --target=$nb_path krwordrank
    !pip install --target=$nb_path heapq
    !pip install --target=$nb_path kss
    !pip install --target=$nb_path bert
    !pip install --target=$nb_path textrankr
    !pip install --target=$nb_path lexrankr
    %cd /content/drive/Shared drives/BigDATA TEAM 10/py-hanspell
    !python setup.py install

In [ ]:
#@title A-Catcher 실행 환경 구축
import os, sys
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/Shared drives/BigDATA TEAM 10
import warnings
warnings.filterwarnings('ignore')
from konlpy.tag import Hannanum, Okt, Komoran
h = Hannanum()
o = Okt()
k = Komoran
import re
import time
import numpy as np
import heapq
from collections import Counter
import pandas as pd
from operator import itemgetter
from collections import deque, defaultdict
from ast import literal_eval
from collections import defaultdict
from pprint import pprint
from krwordrank.word import KRWordRank
from copy import deepcopy
import kss
import itertools
import unicodedata
import requests
from functools import reduce
import torch
from bs4 import BeautifulSoup
import string
from textrankr import TextRank
from lexrankr import LexRank
from hanspell import spell_checker  

!pip install --upgrade google-cloud-vision
!apt-get install -y poppler-utils

%cd /content/drive/'Shared drives'/'BigDATA TEAM 10'
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, request, redirect, url_for
from flask_admin.contrib.fileadmin import FileAdmin
from flask_admin import Admin
from flask_dropzone import Dropzone
from pdf2image import convert_from_path
import io
import os
from google.cloud import vision
from google.cloud.vision import types
import logging
from pptx import Presentation
from pptx_tools import utils

%cd /content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction/frameBERT
!pip install transformers
import frame_parser
path="/content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction/frameBERT"
parser = frame_parser.FrameParser(model_path=path, language='ko')


Mounted at /content/drive
/content/drive/Shared drives/BigDATA TEAM 10
     |████████████████████████████████| 440kB 9.2MB/s 


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 11 not upgraded.
Need to get 154 kB of archives.
After this operation, 613 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 poppler-utils amd64 0.62.0-2ubuntu2.10 [154 kB]
Fetched 154 kB in 0s (2,017 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 144599 files and directories currently installed.)
Preparing to unpack .../poppler-utils_0.62.0-2ubuntu2.10_amd64.deb ...
Unpacking poppler-utils (0.62.0-2ubuntu2.10) ...
Setting up poppler-utils (0.62.0-2ubuntu2.10) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
/content/drive/Shared drives/BigDATA TEAM 10
/content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction/frameBERT
     |████████████████████████████████| 890kB 9.0MB/s

In [ ]:
#@title 자연어 처리 모듈
class Text():
    def __init__(self, text):
        text = re.sub("'", ' ', text)
        paragraphs = text.split('\n')
        self.text = text
        self.paragraphs = [i for i in paragraphs if i]
        self.counts = len(self.paragraphs)
        self.docs = [kss.split_sentences(paragraph) for paragraph in paragraphs if kss.split_sentences(paragraph)]
        self.newtext = deepcopy(self.text)
        print("TEXT")

    def findall(self, p, s):
        i = s.find(p)
        while i != -1:
            yield i
            i = s.find(p, i + 1)
      
    def countMatcher(self, sentences, paragraph_no):
        paragraph = self.docs[paragraph_no]
        total_no = len(paragraph)
        vec = [0] * total_no
        
        for idx, candidate in enumerate(paragraph):
            for sentence in sentences:
                if sentence[:4] in candidate:
                    vec[idx] += 1
        print("Vec ",vec)
        return vec

class Highlight(Text):
    def __init__(self, text):
        super().__init__(text)
        print("Highlight")
        okt = Okt()
        h = Hannanum()

        wordrank_extractor = KRWordRank(min_count=3, max_length=10)
        print("paragraph ",self.paragraphs)
        self.keywords, rank, graph = wordrank_extractor.extract(self.paragraphs)
        self.path = "/content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction"
        p = []
        kw = []
        for k, v in self.keywords.items():
            p.append(okt.pos(k))
            kw.append(k)
        words = self.text.split(' ')
        s = set()
        keylist = [word for i in kw for word in words if i in word]
        keylist = [i for i in keylist if len(i)>2]
        for i in keylist:
            if len(i)>2:
              s.add(i)
        print("KEYLIST: ",keylist)
        p = [okt.pos(word) for word in s]
        s = set()
        for idx in range(len(p)):
            ls = p[idx]
            for j in range(len(ls)):
                tag = ls[j][1]
                word = ls[j][0]
                if tag == "Noun":
                    s.add(word)
        self.keys = []
        for temp in s:
            self.keys.append(" " + temp)

    
    def add_tags_keywords(self):
        self.candidates = self.keys
        self.newtext = deepcopy(self.text)
        self.idx = [(i, i + len(candidate)) for candidate in self.candidates for i in
                        self.findall(candidate, self.text)]
        for i in range(len(self.idx)):
            try:
                self.idx = [(start, start + len(candidate)) for candidate in self.candidates for start in
                            self.findall(candidate, self.newtext)]
                word = self.newtext[self.idx[i][0]:self.idx[i][1]]
                tagged = " <mark style='background-color:#FFD0F2'>%s</mark>" % (word)
                self.newtext = tagged.join([self.newtext[:self.idx[i][0]], self.newtext[self.idx[i][1]:]])
            except:
                pass
        return self.newtext


    def add_tags_conj(self, txt):
        conj = '그리고, 그런데, 그러나, 그래도, 그래서, 또는, 및, 즉, 게다가, 따라서, 때문에, 아니면, 왜냐하면, 단, 오히려, 비록, 예를 들어, 반면에, 하지만, 그렇다면, 바로, 이에 대해'
        conj = conj.replace("'", "")
        self.candidates = conj.split(",")
        self.newtext = deepcopy(txt)
        self.idx = [(i, i + len(candidate)) for candidate in self.candidates for i in
                        self.findall(candidate, txt)]
        for i in range(len(self.idx)):
            try:
                self.idx = [(start, start + len(candidate)) for candidate in self.candidates for start in
                            self.findall(candidate, self.newtext)]
                word = self.newtext[self.idx[i][0]:self.idx[i][1]]
                res = ""
                tagged = " <mark style='background-color:#F9D877'>%s</mark>" % (word)
                print(tagged)
                self.newtext = tagged.join([self.newtext[:self.idx[i][0]], self.newtext[self.idx[i][1]:]])
            except:
                pass
        return self.newtext

    def highlight(self):
          result = self.add_tags_keywords()
          cleanr = re.compile("</mark>.{0,5}<mark style='background-color:#FFD0F2'>")
          patterns = re.findall("</mark>.{0,5}<mark style='background-color:#FFD0F2'>",result)
          w = [i[7:-39] for i in patterns]
          for i in range(len(w)):
            result = re.sub(patterns[i], w[i], result)
          txt = self.add_tags_conj(result)
          print("Highlighted Result ", txt)
          return txt
class Node:
    def __init__(self, data):
        self.words = data[0]
        self.usage = data[1]
        self.tags = data[3]
        self.next = None

    def __repr__(self):
        return str((self.words, self.tags))


class LinkedList:
  def __init__(self):
      self.head = None

  def __repr__(self):
      node = self.head
      nodes = []
      while node is not None:
          nodes.append(str(node.tags))
          node = node.next
      nodes.append("None")
      return ' -> '.join(nodes)


class Relation(object):
  def __init__(self):
      self.path="/content/drive/Shared drives/BigDATA TEAM 10/OpenInformationExtraction"
      self.parser = parser

  def findConsecutiveBIO(self, words, tag):
      began = False
      count = 1
      que = " "
      temp_word = []
      temp_tag = []
      res = " "

      for i in range(len(words)):
          if not tag[i].startswith("O"):
              began = True
              temp_word.append(words[i])
              temp_tag.append(tag[i].split("-")[1])
          elif tag[i].startswith("O"):
              began = False
      r = que.join(temp_word)
      s = que.join(temp_tag)
      return r, s

  def relationPopup(self, sent):
      parsed = parser.parser(sent, sent_id='1', result_format='conll')
      parsedList = LinkedList()
      self.final = dict()
      for j in range(len(parsed)):
          parsed_candidate = parsed[j]
          print(" Parsed Candidate ", parsed_candidate)
          new_node = Node(parsed_candidate)
          if j == 0:
              old_node = new_node
              parsedList.head = old_node
          elif j == len(parsed) - 1:
              old_node.next = new_node
              new_node.next = None
              print(j, '  ', parsedList)
              self.final[j] = parsedList
          else:
              old_node.next = new_node
              old_node = new_node
      res = dict()

      for k, v in self.final.items():
          a = v.head
          while a is not None:
              i = 0
              temp = " "
              if len(a.words) == len(a.tags):
                  w, t= self.findConsecutiveBIO(a.words, a.tags)
                  res[t]=w
              else:

                  print("ERROR OCCURED")
              a = a.next

      return res
      
  def arrow(self, words, tags):
      d = defaultdict(set)
      words = words.split(" ")
      tags = tags.split(" ")
      total = ''
      tag = ''
      for idx, relation in enumerate(tags):
        print(relation)
        d[relation].add(idx)
      res = ""
      for i, s in d.items():
        print(s)
        temp = []
        for idx in s:
          w = okt.pos(words[idx])
          print("word",w)
          if  w == "Noun" or w == "Adjective":
            temp.append(words[idx])
        total += ' '.join(temp)
        total += "➜"
        tag += i
        tag += "➜"
      return total[:-1], tag[:-1]

class Summarize(Highlight):
    def __init__(self, text, paragraph_no):
      super().__init__(text)
      print("length of paragraphs ",len(self.paragraphs))
      self.txt = self.paragraphs[paragraph_no]
      self.paragraph_no = paragraph_no

    def summarize(self):
        url = "https://api.smrzr.io/summarize?ratio=0.15"
        headers = {
            'content-type': 'raw/text',
            'origin': 'https://smrzr.io',
            'referer': 'https://smrzr.io/',
            'sec-fetch-dest': 'empty',
            'sec-fetch-mode': 'cors',
            'sec-fetch-site': 'same-site',
            "user-agent": "Mozilla/5.0"
        }
        resp = requests.post(url, headers=headers, data= self.txt.encode('utf-8'))
        assert resp.status_code == 200
        summary = resp.json()['summary']
        temp = summary.split('\n')
        print("BERT: ", temp)
        return temp


    def summarizeTextRank(self, max=1):
        tr = TextRank(self.txt)
        
        summary = tr.summarize(max).split('\n')
        print("Textrank: ",summary)
        return summary


    def summarizeLexRank(self):
        lr = LexRank()
        lr.summarize(self.txt)
        summaries = lr.probe()
        print("Lexrank: ",summaries)
        return summaries

    def ensembleSummarize(self):
        a = np.array(self.countMatcher(self.summarize(), self.paragraph_no))
        
        try:
          b = np.array(self.countMatcher(self.summarizeLexRank(), self.paragraph_no))
        except:
          b = np.zeros_like(a)
        c = np.array(self.countMatcher(self.summarizeTextRank(),self.paragraph_no))
        result= a+b+c
        i, = np.where(result == max(result))
        txt, index = self.docs[self.paragraph_no][i[0]], i[0]
        return txt, index

####################################################################################################


import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/drive/Shared drives/BigDATA TEAM 10/dataCampusProject-Team10/ocr/My Project-a3f9a30ceabf.json"
client = vision.ImageAnnotatorClient()

In [ ]:
#@title Flask 작동 코드
%cd /content/drive/'Shared drives'/'BigDATA TEAM 10'/dataCampusProject-Team10/flask
app = Flask(__name__)
run_with_ngrok(app)

basedir = '/content/drive/Shared drives/BigDATA TEAM 10/dataCampusProject-Team10/flask' 
upload_dir = os.path.join(basedir, 'uploads')  # basedir 의 uploads 에 파일 저장
#################################################################################################################

admin = Admin(name='Uploaded Files')
admin.init_app(app)
dropzone = Dropzone(app)
admin.add_view(FileAdmin(upload_dir, name='FILES')) 
app.config['DROPZONE_ALLOWED_FILE_CUSTOM'] = True
app.config['DROPZONE_ALLOWED_FILE_TYPE'] = 'image/*, .pdf, .txt'
app.config['DROPZONE_REDIRECT_VIEW']='result'


#############################################################################
@app.route("/", methods=['GET', 'POST'])
def upload(num=None):
    if request.method == 'POST':
        f = request.files.get('file')
        f.save(os.path.join(upload_dir, f.filename))
        images = convert_from_path(os.path.join(upload_dir, f.filename))
        for i, image in enumerate(images):
            fname = "uploads/image" + str(i) + ".jpg"
            image.save(fname, "JPEG")
        # return redirect(url_for('result'))
    return render_template('homepage.html',num=num)


###############################################################################
@app.route("/result", methods=['GET', 'POST'])
def result(num=None):
    # if request.method == 'POST':
    with io.open("/content/drive/Shared drives/BigDATA TEAM 10/dataCampusProject-Team10/flask/uploads/image0.jpg",
                  'rb') as image_file:
        content = image_file.read()
    image = vision.types.Image(content=content)
    response = client.text_detection(image=image)
    texts = response.text_annotations
    box = 0, 0, 0, 0, 0, 0, 0, 0
    pre_box = 0, 0, 0, 0, 0, 0, 0, 0
    result_text = []
    a = ' '
    for text in texts[1:]:
        for vertex in text.bounding_poly.vertices:
            vertices = (['{},{}'.format(vertex.x, vertex.y) for vertex in text.bounding_poly.vertices])
            x1, x2, x3, x4 = vertices[0].split(',')[0], vertices[1].split(',')[0], vertices[2].split(',')[0], \
                              vertices[3].split(',')[0]
            y1, y2, y3, y4 = vertices[0].split(',')[1], vertices[1].split(',')[1], vertices[2].split(',')[1], \
                              vertices[3].split(',')[1]
            break

        box = int(x1), int(y1), int(x2), int(y2), int(x3), int(y3), int(x4), int(y4)

        if (box[1] - pre_box[5]) >= 30:
            result_text.append(a)
            a = text.description
        else:
            a += text.description + ' '
        pre_box = box
    result_text.append(a)
    print(" 🧚‍♀️ OCR running 🧚‍♀️ ")
    
    b = "\n"
    b = b.join(result_text)
    with open("/content/result_text.txt", 'w') as f:
      f.write(b)
    
    txt = ""

    with open("/content/result_text.txt") as f:
      while True:
        line = f.readline()
        line = spell_checker.check(line)[2]
        print("Checked spelling ",line)
        txt += line
        txt += "\n"
        if not line: 
          break
        
    txt = [i.replace("딸", "말") for i in txt.split("\n") if i]


    print(" 😍  NLP running  😍 ")
    d = [kss.split_sentences(i) for i in txt if i]
    rel = Relation()
    pu = []
    for paragraph in range(len(d)):
        a = []
        for i in range(len(d[paragraph])):
            a.append(rel.relationPopup(d[paragraph][i]))
        pu.append(a)

    pop_doc=[]
    for p in range(len(pu)):
        dct = [""]*len(pu[p])
        for s in range(len(pu[p])):
            counter = 0
            for k, v in pu[p][s].items():
                z = k.split(" ")
                if len(Counter(z))>counter and 8>len(Counter(z))>1:
                    dct[s]=(str(v)+":"+str(k))
                counter = len(Counter(z))
        pop_doc.append(dct)
    print("Relation Extracted")
    rs = ""
    for p in range(len(pop_doc)):
        pop = [i for i in pop_doc[p] if i]
        for r in pop:
            temp = r.split(":")
            s = ""
            sent = ""
            tagged = temp[1].split(" ")
            words = temp[0].split(" ")
            curr = tagged[0]
            print(tagged, words, curr)
            for i, tag in enumerate(tagged):
                if curr not in tag:
                      s+="▶ "
                      sent+="▶ "
                s+=(" "+tag)
                sent+=(" "+words[i])
                curr = tag
            rs += "<a data-toggle='tooltip' data-placement='top' title="+ str(s) + ">" + str(sent) +"</a>"+"<br>"
        rs+='<\n\n>'
    with open("/content/drive/Shared drives/BigDATA TEAM 10/dataCampusProject-Team10/flask/relation.txt",'w') as file:
        file.write(rs)


    result = '\n'.join(txt)
    high = Highlight(result)
    tagged = high.highlight()
    paragraphs = [i for i in tagged.split('\n') if i]
    total = ''
    for idx in range(len(paragraphs)):
      
      summarizer = Summarize(result, idx)
      txt, id = summarizer.ensembleSummarize()
      print("summarized ",txt)
      paragraph = kss.split_sentences(paragraphs[idx])
      sent_counts = len(paragraph)
      for sent in range(sent_counts):
        if txt[:3] in paragraph[sent]:
          relation = ""
          string ="<u style='text-decoration:underline; text-decoration-color:#906fa8; font-weight: bold; text-decoration-style: wavy'>" + paragraph[sent] +"</u>"
          total += string
        else:
          total += paragraph[sent]
      if idx is not len(paragraphs)-1:
        total += "\n"
    with open("/content/drive/Shared drives/BigDATA TEAM 10/dataCampusProject-Team10/flask/result.txt",'w') as file:
        file.write(total)
    
    utils.save_pptx_as_png("/content/drive/'Shared drives'/'BigDATA TEAM 10'/dataCampusProject-Team10/flask/outputs", "/content/drive/'Shared drives'/'BigDATA TEAM 10'/dataCampusProject-Team10/flask/slides.pptx", overwrite_folder=True)

    with open(os.path.join(basedir, 'result.txt'), 'r', encoding="UTF-8") as file:
        string = file.readlines()
    with open(os.path.join(basedir, "relation.txt"), 'r', encoding="UTF-8") as file:
        relation = file.read()
    relation = [i for i in relation.split('\n\n\n') if i]
    print(relation, len(relation))
    print(string, len(string))
    with open(os.path.join(basedir, "templates/result_1.html"), "r", encoding="UTF-8") as file:
        result_1 = file.read()
    with open(os.path.join(basedir, "templates/result_1_1.html"), "r", encoding="UTF-8") as file:
        result_1_1 = file.read()
    with open(os.path.join(basedir, "templates/result_1_2.html"), "r", encoding="UTF-8") as file:
        result_1_2 = file.read()
    with open(os.path.join(basedir, "templates/result_1_3.html"), "r", encoding="UTF-8") as file:
        result_1_3 = file.read()
    with open(os.path.join(basedir, "templates/result_1_4.html"), "r", encoding="UTF-8") as file:
        result_1_4 = file.read()
    with open(os.path.join(basedir, "templates/result_1_5.html"), "r", encoding="UTF-8") as file:
        result_1_5 = file.read()
    with open(os.path.join(basedir, "templates/result_1_6.html"), "r", encoding="UTF-8") as file:
        result_1_6 = file.read()
    

    for j in range(1, len(string)):
        result_1 += ("<a class='nav-link waves-effect' href='#PG"+ str(j) + "'><mark><strong>Paragraph "+ str(j) + "</strong></mark></a>")         
    result_1 += ("<a class='nav-link waves-effect' href='#PP'><mark><strong>PowerPoint Slides</strong></mark></a>")

    for i in range(1, len(string)):
        result_1 += ("<div class='col-md-12' id='PG" + str(i) + "'><hr><div class='row-eq-height'><div class='col-md-12' style='line-height: 40px'><p class='dark-grey-text'><mark><strong>Paragraph" + str(i) + "</strong></mark></p><a style='line-height: 40px'>")
        result_1 += string[i] # paragraph
        result_1 += ("</a><br><div class='col-md-12 text-right'><button type='button' class='btn-yellow' data-toggle='modal' data-target='#PPT" + str(i) + "'>PPT</button><button type='button' class='btn-deep-orange' data-toggle='modal' data-target='#P" + str(i) + "'>Relation</button></div><div class='modal' id='PPT" + str(i) + "'>")
        result_1 += result_1_1
        result_1 += ("<img src='https://raw.githubusercontent.com/yoonkim313/dataCampusProject-Team10/master/5.%20Flask/slide" + str(i) + ".PNG' class='img-fluid' alt='Sample post image'>")  # ppt slide picture
        result_1 += result_1_2
        result_1 += ("<div class='modal' id='P" + str(i) + "'>")  # relation tagging modal
        result_1 += result_1_3
        try: result_1 += relation[i-1]
        except: pass
        result_1 += result_1_4
    
    result_1 += result_1_5
    result_1 += ("<div class='carousel-item active'><img src='https://raw.githubusercontent.com/yoonkim313/dataCampusProject-Team10/master/5.%20Flask/slide1.PNG' class='d-block w-100' alt='...'></div>")
    
    if len(string) > 1:
        for i in range(2, len(string)):
          result_1 += ("<div class='carousel-item'><img src='https://raw.githubusercontent.com/yoonkim313/dataCampusProject-Team10/master/5.%20Flask/slide" + str(i) + ".PNG' class='d-block w-100' alt='...'></div>")
    
    result = result_1 + result_1_6

    with open(os.path.join(basedir, "templates/final_result.html"), "w", encoding="UTF-8") as file:
        file.write(result)

    return render_template('final_result.html')
#################################################################################################################


if __name__ == '__main__':
    app.run()

/content/drive/Shared drives/BigDATA TEAM 10/dataCampusProject-Team10/flask
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://f03ec9dd79e4.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [14/Sep/2020 15:43:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2020 15:43:14] "GET /static/css/bootstrap.min.css HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2020 15:43:14] "GET /static/css/mdb.min.css HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2020 15:43:14] "GET /static/css/style.min.css HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2020 15:43:15] "GET /static/js/jquery-3.4.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2020 15:43:15] "GET /static/js/popper.min.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2020 15:43:15] "GET /static/js/bootstrap.min.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2020 15:43:15] "GET /static/js/mdb.min.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2020 15:43:15] "GET /static/img/svg/arrow_right.svg HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2020 15:43:15] "GET /static/img/svg/arrow_left.svg HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2020 15:43:16] "GET /static/font/roboto/Roboto-Light.woff2 HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2020 15:43:16] "GET /static/font/roboto/Roboto-Reg

 🧚‍♀️ OCR running 🧚‍♀️ 
Checked spelling   
Checked spelling  암세포의 대사 과정 은 정상 세포 와 다른 것으로 알려져 있다. 오토 바르 부르크 가 발표 한 ' 바르 부르크 효과 '에 따르면 암세포는 ' 해당 작용 ' 을 주된 에너지 획득 기전으로 수행하고 또 다른 에너 지 획득 방법인 ' 산화 적 인산화 '는 억제한다. 
Checked spelling  세포는 영양분으로 섭취 한 큰 분자를 작은 분자로 쪼개는 과정을 통해 ATP를 생성하는데 이 과정 은 ' 이화 작용 '이라고 한다. 또한 ATP 와 같은 고 에너지 분자의 에너지를 이용하여 세포의 성장과 분열을 위해 작은 분자로부터 단백질, 핵 신과 같은 거대 분자를 합성하는 과정을 ' 동화 작용 ' 이라고 한다. 이화 작용을 통해 ATP를 생산하기 위해 세포는 영양물질을 내부로 수송하는데, 가장 대표적인 영양물질인 포도당 은 세포 내부로 이동하여 해당 작용과 산화 적 인산화를 통해 작은 분 자로 분해된다. 이론적으로 포도당 1 개가 가지고 있는 에너지 가 전부 ATP로 전환될 경우 36 개 또 
Checked spelling  는 38 개의 ATP 가 만들어진다. 이 중 2 개의 ATP는 세포질에서 일어나는 해당 작용을 통해, 나머지는 미토콘드리아에서 대부분 산화 적 인산화를 통해 만들어진다. 
Checked spelling  
 😍  NLP running  😍 
 Parsed Candidate  [['암세포의', '대사', '과정', '은', '정상', '세포', '와', '다른', '것으로', '알려져', '있다.'], ['_', '_', '과정.n', '_', '_', '_', '_', '_', '_', '_', '_'], ['_', '_', 'Process', '_', '_', '_', '_', '_', '_', '_', '_'], ['B-Process', 'I-Process', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '

127.0.0.1 - - [14/Sep/2020 15:43:38] "GET /result HTTP/1.1" 200 -


BERT:  ['이 중 2 개의 ATP는 세포질에서 일어나는 해당 작용을 통해 , 나머지는 미토콘드리아에서 대부분 산화 적 인산화를 통해 만들어진다 .']
Vec  [0, 1]
Lexrank:  ['는 38 개의 ATP 가 만들어진다', '이 중 2 개의 ATP는 세포질에서 일어나는 해당 작용을 통해, 나머지는 미토콘드리아에서 대부분 산화 적 인산화를 통해 만들어진다']
Vec  [1, 1]
Textrank:  ['는 38 개의 ATP 가 만들어진다.']
Vec  [1, 0]
summarized  는 38 개의 ATP 가 만들어진다.
Comptype module needed to save PNGs.
["<\n\n><a data-toggle='tooltip' data-placement='top' title= Agent▶  Act Act Act Act Act Act> 암세포는▶  ' 작용 주된 에너지 획득 기전으로</a><br><\n\n><a data-toggle='tooltip' data-placement='top' title= Creator▶  Created_entity> 세포는▶  ATP를</a><br><a data-toggle='tooltip' data-placement='top' title= Means Means Means Means Means Means Means Means▶  Goal Goal Goal▶  Means Means Means> ATP 와 같은 고 에너지 분자의 에너지를 이용하여▶  세포의 성장과 분열을▶  작은 분자로부터 단백질,</a><br><a data-toggle='tooltip' data-placement='top' title= Agent▶  Means Means Means Means> 세포는▶  작용과 산화 적 인산화를</a><br><a data-toggle='tooltip' data-placement='top' title= Entity Entity Entity▶  Final_category Final_category> 포도당 1